## 1. Setup and Import Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print('✓ Libraries imported successfully')

✓ Libraries imported successfully


## 2. Load the Data
- KEEP AS EXCEL SHEET BECAUSE OF THE FILE SIZE WHICH ISN'T VERY BIG ; OTHERWISE CSV FILES ARE FASTER 

In [9]:
# Load the Excel file
file_path = r"C:\Users\sabah\OneDrive\Desktop\trendyol_case\data\Interview Case - Language Data Analyst - For candidates.xlsx"
# Check all sheets
excel_file = pd.ExcelFile(file_path)
print(f"Sheets available: {excel_file.sheet_names}")

# Load the Data sheet
df = pd.read_excel(file_path, sheet_name='Data')
print(f"\n✓ Data loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")

Sheets available: ['Intro', 'Case', 'Data']

✓ Data loaded: 17,898 rows × 18 columns


## 3. Initial Data Exploration

In [10]:
# Display first few rows
print("First 3 rows of data:")
df.head(3)

First 3 rows of data:


,ctmsId,externalId,namespace,contentType,createdAt,sourceLanguage,sourceText,targetLanguage,enReferenceTranslation,targetText,contentId,translationProvider,productViewCount,productRevenue,productURL,Evaluation,Root Cause,Comment
0,prod-qna_prod-qna_382773544_264427545_a,382773544_264427545_a,prod-qna,prod-qna,2024-11-10 18:21:16.242393 UTC,tr-tr,"Merhaba, ürünün bütün bedenler için iç boy ölç...",hu-hu,"Hello, the inner length of the product for all...","Sziasztok, a termék belső hossza minden méretn...",382773544,GoogleTranslate,314560.0,83568,https://www.trendyol.com/hu/pname-p-382773544,OK,NaN,NaN
1,prod-qna_prod-qna_729116297_287101008_a,729116297_287101008_a,prod-qna,prod-qna,2025-01-29 06:02:54.223384 UTC,tr-tr,"Merhaba, modelin boyu 173 cm. Vücut ölçüleri b...",hu-hu,"Hello, the model's height is 173 cm. Body meas...","Hello, a modell magassága 173 cm. A testmérete...",729116297,GoogleTranslate,159493.0,50109,https://www.trendyol.com/hu/pname-p-729116297,OK,NaN,NaN
2,prod-qna_prod-qna_845564735_262708650_a,845564735_262708650_a,prod-qna,prod-qna,2024-11-05 09:24:28.002988 UTC,tr-tr,"Merhaba, ilgili sorularınızı Hesabım-Trendyol ...",hu-hu,"Hello, we kindly ask you to submit your releva...","Üdvözöljük! Kérjük, tegye fel kérdéseit a Fiók...",845564735,GoogleTranslate,112840.0,51696,https://www.trendyol.com/hu/pname-p-845564735,OK,NaN,NaN


In [13]:
# Column information
print("Dataset Columns and Data Types:")
print("="*50)
for col in df.columns:
    non_null = df[col].notna().sum()
    null_pct = (df[col].isna().sum() / len(df)) * 100
    print(f"{col:<25} {str(df[col].dtype):<10} {non_null:>6}/{len(df)} ({null_pct:>5.1f}% missing)")

Dataset Columns and Data Types:
ctmsId                    object      17798/17898 (  0.6% missing)
externalId                object      17798/17898 (  0.6% missing)
namespace                 object      17898/17898 (  0.0% missing)
contentType               object      17898/17898 (  0.0% missing)
createdAt                 object      17372/17898 (  2.9% missing)
sourceLanguage            object      17898/17898 (  0.0% missing)
sourceText                object      17871/17898 (  0.2% missing)
targetLanguage            object      17898/17898 (  0.0% missing)
enReferenceTranslation    object      13382/17898 ( 25.2% missing)
targetText                object      17898/17898 (  0.0% missing)
contentId                 object      17219/17898 (  3.8% missing)
translationProvider       object      12989/17898 ( 27.4% missing)
productViewCount          float64     10184/17898 ( 43.1% missing)
productRevenue            int64       17898/17898 (  0.0% missing)
productURL                obje

In [14]:
# Basic statistics
print("\n📊 DATASET OVERVIEW")
print("="*50)
print(f"Total entries: {len(df):,}")
print(f"\nSource language: {df['sourceLanguage'].unique()}")
print(f"\nTarget languages ({df['targetLanguage'].nunique()}):")
for lang in sorted(df['targetLanguage'].unique()):
    count = (df['targetLanguage'] == lang).sum()
    print(f"  • {lang}: {count:,} entries")

print(f"\nTranslation providers ({df['translationProvider'].nunique()}):")
for provider in df['translationProvider'].value_counts().index:
    count = (df['translationProvider'] == provider).sum()
    print(f"  • {provider}: {count:,} entries")


📊 DATASET OVERVIEW
Total entries: 17,898

Source language: ['tr-tr']

Target languages (11):
  • ar-ae: 1,600 entries
  • bg-bg: 1,600 entries
  • cs-cz: 1,600 entries
  • de-de: 1,600 entries
  • el-gr: 1,600 entries
  • en-us: 2,079 entries
  • hu-hu: 1,507 entries
  • pl-pl: 1,562 entries
  • ro-ro: 1,600 entries
  • sk-sk: 1,600 entries
  • uk-ua: 1,550 entries

Translation providers (5):
  • Alibaba: 6,368 entries
  • GoogleTranslate: 3,903 entries
  • DeepL: 2,570 entries
  • ctms-translation-validation: 139 entries
  • GoogleAutoML: 9 entries


## 4. Data Cleaning

In [15]:
# Check for data quality issues
print("Data Quality Issues Found:")
print("="*50)

# Check Evaluation column for inconsistencies
print("\n1. Evaluation column values:")
print(df['Evaluation'].value_counts())

# Fix typos
df['Evaluation'] = df['Evaluation'].replace({
    'Not ok': 'Not OK',
    'Evvaluation Blocked': 'Evaluation Blocked'
})

print("\n✓ Fixed typos in Evaluation column")
print("\nCleaned Evaluation values:")
print(df['Evaluation'].value_counts())

Data Quality Issues Found:

1. Evaluation column values:
Evaluation
OK                     10017
Ideal                   3727
Not OK                  2965
Evaluation Blocked       475
Evvaluation Blocked        1
Not ok                     1
Name: count, dtype: int64

✓ Fixed typos in Evaluation column

Cleaned Evaluation values:
Evaluation
OK                    10017
Ideal                  3727
Not OK                 2966
Evaluation Blocked      476
Name: count, dtype: int64


## 5. Quality Metrics Calculation

In [16]:
# Overall quality distribution
print("📈 QUALITY DISTRIBUTION")
print("="*50)

quality_dist = df['Evaluation'].value_counts()
quality_pct = (quality_dist / len(df) * 100).round(1)

for eval_type, count in quality_dist.items():
    pct = quality_pct[eval_type]
    bar = '█' * int(pct/2)  # Visual bar
    print(f"{eval_type:<20} {count:>6} ({pct:>5.1f}%) {bar}")

# Calculate error rate
evaluated = df[df['Evaluation'].isin(['OK', 'Not OK', 'Ideal'])].shape[0]
not_ok = df[df['Evaluation'] == 'Not OK'].shape[0]
error_rate = (not_ok / evaluated) * 100

print(f"\n🎯 KEY METRIC: Overall Error Rate = {error_rate:.1f}%")
print(f"   (Calculated as Not OK / Total Evaluated)")

📈 QUALITY DISTRIBUTION
OK                    10017 ( 56.0%) ████████████████████████████
Ideal                  3727 ( 20.8%) ██████████
Not OK                 2966 ( 16.6%) ████████
Evaluation Blocked      476 (  2.7%) █

🎯 KEY METRIC: Overall Error Rate = 17.7%
   (Calculated as Not OK / Total Evaluated)


In [17]:
# Quality Score Calculation
# Weighted score: Ideal = 1.5, OK = 1.0, Not OK = 0

ideal_count = (df['Evaluation'] == 'Ideal').sum()
ok_count = (df['Evaluation'] == 'OK').sum()
not_ok_count = (df['Evaluation'] == 'Not OK').sum()

quality_score = ((ideal_count * 1.5) + (ok_count * 1.0)) / evaluated

print(f"\n📊 QUALITY SCORE: {quality_score:.1%}")
print(f"   Calculation: (Ideal×1.5 + OK×1.0) / Total Evaluated")
print(f"   • Ideal: {ideal_count:,} × 1.5")
print(f"   • OK: {ok_count:,} × 1.0")
print(f"   • Not OK: {not_ok_count:,} × 0")


📊 QUALITY SCORE: 93.4%
   Calculation: (Ideal×1.5 + OK×1.0) / Total Evaluated
   • Ideal: 3,727 × 1.5
   • OK: 10,017 × 1.0
   • Not OK: 2,966 × 0


## 6. Save Cleaned Data

In [18]:
# Save cleaned data for next steps
df.to_csv('../outputs/cleaned_data.csv', index=False)
print("✓ Cleaned data saved to: ../outputs/cleaned_data.csv")
print(f"  Shape: {df.shape}")
print(f"  File ready for Part 2 analysis")

✓ Cleaned data saved to: ../outputs/cleaned_data.csv
  Shape: (17898, 18)
  File ready for Part 2 analysis


## Summary

### ✅ Part 1 Completed!

**Key Findings:**
- Total entries: 17,898
- Overall error rate: 17.7%
- Quality score: 93.4%
- 11 target languages
- 5 translation providers

**Next Steps:**
- Part 2: Pattern Analysis by Language/Provider
- Part 3: Visualization Dashboard
- Part 4: Root Cause Analysis
- Part 5: Technical Solution Design